# Task #03 作業 02【實作題】將資料存入資料庫
#練習：在前述的練習中，你已經利用爬蟲成功的收集到了來自台灣銀行牌告匯率的資料，但我們現在每天只會抓取到一筆當日的資料。如果我們希望能夠觀察趨勢的話，會希望能夠週期性的抓取並且儲存下來。接下來這個任務是請你將這個結果紀錄再加上更新時間的欄位，將資料存到到 SQLite 資料庫（名稱欄位自訂）中。

❏ Sample Input：https://rate.bot.com.tw/xrt?Lang=zh-TW（資料來源：台灣銀行牌告匯率 ）

❏ Sample Output：將匯率資訊加上更新更新時間欄位並且將資料存到到 SQLite 資料庫中

❏ Sample Code：

'''
Your Code
'''

In [1]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup

In [2]:
# request data from website
url = 'https://rate.bot.com.tw/xrt?Lang=zh-TW'
r = requests.get(url)

d = BeautifulSoup(r.text)
print(d)

<!DOCTYPE html>
<html class="no-js" lang="zh-TW">
<head>
<meta charset="utf-8"/>
<title>臺灣銀行牌告匯率</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="臺灣銀行匯率利率黃金牌價查詢" name="description"/>
<meta content="" name="keywords"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="https://www.bot.com.tw/assets/images/bot-og-image_480x480.png" property="og:image"/>
<meta content="index,follow" name="robots"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/Content/css/font-awesome.min.css" rel="stylesheet"/>
<link href="/Content/css/jquery-ui.min.css" rel="stylesheet"/>
<link href="/Content/main-3.8.0.css" rel="stylesheet"/>
<style nonce="xvMQgc4Yb2uDczRQN3upLS9Pi/GpgkD4Uch9gtSJTiQ=">
    h1.hero__header { font-size:1.5625rem }
    
    
    .bank {display:none}
    </style>
<noscript>
<style>
            table th.noscript {
                d

In [3]:
# find all exchange rate by html class
data = d.find_all('td',class_="text-right display_none_print_show print_width")

In [4]:
# turn the exchange rate data to different array
df1 = [] #現金匯率本行買入
df2 = [] #現金匯率本行賣出
df3 = [] #即期匯率本行買入
df4 = [] #即期匯率本行賣出
for i in range(len(data)):
    #print(data[i])
    #print(data[i].string)
    #print(data[i]['data-table'])
    if data[i]['data-table'] == '本行現金買入':
        if data[i].string == '-':
            df1.append(data[i].string)
        else:
            df1.append(float(data[i].string))
    elif data[i]['data-table'] == '本行現金賣出':
        if data[i].string == '-':
            df2.append(data[i].string)
        else:
            df2.append(float(data[i].string))
    elif data[i]['data-table'] == '本行即期買入':
        if data[i].string == '-':
            df3.append(data[i].string)
        else:
            df3.append(float(data[i].string))
    elif data[i]['data-table'] == '本行即期賣出':
        if data[i].string == '-':
            df4.append(data[i].string)
        else:
            df4.append(float(data[i].string))

In [5]:
# deal with the currency
df0 = []
data_cur = d.find_all('div',class_="hidden-phone print_show xrt-cur-indent")
for i in range(len(data_cur)):
    df0.append(data_cur[i].string.strip())

In [15]:
# search the update time
time = d.find('span',class_="time").string
df_time = [time]*len(data_cur)

['2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25',
 '2024/05/11 09:25']

In [16]:
# create dataframe of data
data = pd.DataFrame({
    '幣別':df0,
    '現金匯率本行買入':df1,
    '現金匯率本行賣出':df2,
    '即期匯率本行買入':df3,
    '即期匯率本行賣出':df4,
    '更新時間':df_time
})
data

,幣別,現金匯率本行買入,現金匯率本行賣出,即期匯率本行買入,即期匯率本行賣出,更新時間
0,美金 (USD),32.01,32.68,32.335,32.485,2024/05/11 09:25
1,港幣 (HKD),3.993,4.197,4.114,4.184,2024/05/11 09:25
2,英鎊 (GBP),39.4,41.52,40.295,40.925,2024/05/11 09:25
3,澳幣 (AUD),21.01,21.79,21.225,21.57,2024/05/11 09:25
4,加拿大幣 (CAD),23.22,24.13,23.55,23.88,2024/05/11 09:25
5,新加坡幣 (SGD),23.36,24.27,23.83,24.05,2024/05/11 09:25
6,瑞士法郎 (CHF),34.95,36.15,35.56,35.95,2024/05/11 09:25
7,日圓 (JPY),0.199,0.2118,0.2058,0.2108,2024/05/11 09:25
8,南非幣 (ZAR),-,-,1.712,1.802,2024/05/11 09:25
9,瑞典幣 (SEK),2.6,3.12,2.93,3.05,2024/05/11 09:25


In [17]:
# 建立 SQLite 資料庫連線
conn = sqlite3.connect('test.db')

In [19]:
# 將 DataFrame 的資料寫入資料庫
data.to_sql('ExchangeRate', conn, if_exists='append', index=False)

# 關閉資料庫連線
conn.close()

In [28]:
# Check DB
# 建立 SQLite 資料庫連線
conn = sqlite3.connect('test.db')

# 讀取資料表資料
df = pd.read_sql_query("SELECT * from ExchangeRate", conn)

# 關閉資料庫連線
conn.close()

# 顯示 DataFrame
df

,幣別,現金匯率本行買入,現金匯率本行賣出,即期匯率本行買入,即期匯率本行賣出,更新時間
0,美金 (USD),32.01,32.68,32.335,32.485,2024/05/11 09:25
1,港幣 (HKD),3.993,4.197,4.114,4.184,2024/05/11 09:25
2,英鎊 (GBP),39.4,41.52,40.295,40.925,2024/05/11 09:25
3,澳幣 (AUD),21.01,21.79,21.225,21.57,2024/05/11 09:25
4,加拿大幣 (CAD),23.22,24.13,23.55,23.88,2024/05/11 09:25
5,新加坡幣 (SGD),23.36,24.27,23.83,24.05,2024/05/11 09:25
6,瑞士法郎 (CHF),34.95,36.15,35.56,35.95,2024/05/11 09:25
7,日圓 (JPY),0.199,0.2118,0.2058,0.2108,2024/05/11 09:25
8,南非幣 (ZAR),-,-,1.712,1.802,2024/05/11 09:25
9,瑞典幣 (SEK),2.6,3.12,2.93,3.05,2024/05/11 09:25
